<a href="https://colab.research.google.com/github/ebgonzal/notebooks/blob/master/HELPER_CHALLENGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#### This notebook is aimed to help candidates to implement a new solution to detect disturbance and recovey on forest vegetation


**📚**

## Import all required modules

In [ ]:
import ee
import geemap

## Config variables

In [ ]:

Map = geemap.Map()

SENSOR = 'LANDSAT' # LANDSAT | SENTINEL

# if you set true the ndfi values will be rescaled to 1-200
USE_INT_VALUES = False


PERIOD_T0 = ['2020-05-01', '2020-10-30']
PERIOD_T1 = ['2022-05-01', '2022-10-30']


# just an example
YOUR_ROI = ee.Geometry.Polygon([
    [
      [
        -47.68676230217102,
        -3.183132014269646
      ],
      [
        -47.501368015061644,
        -3.183132014269646
      ],
      [
        -47.501368015061644,
        -3.0000645687350738
      ],
      [
        -47.68676230217102,
        -3.0000645687350738
      ],
      [
        -47.68676230217102,
        -3.183132014269646
      ]
    ]
])






# only for SENTINEL
ASSET_SENTINEL = 'COPERNICUS/S2_HARMONIZED'
ASSET_CLOUD_PROB = 'GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED'

BANDS = [
    'B2',
    'B3',
    'B4',
    'B8',
    'B11',
    'B12'
]

NEW_BAND_NAMES = [
    'blue',
    'green',
    'red',
    'nir',
    'swir1',
    'swir2'
]


# Use 'cs' or 'cs_cdf', depending on your use case; see docs for guidance.
# source: https://code.earthengine.google.com/?scriptPath=Examples%3ADatasets%2FGOOGLE%2FGOOGLE_CLOUD_SCORE_PLUS_V1_S2_HARMONIZED
QA_BAND = 'cs_cdf';

CLEAR_THRESHOLD = 0.60

## Cloud Collection for Sentinel

In [ ]:
cloud_prob_sentinel = ee.ImageCollection(ASSET_SENTINEL)

## Helper functions

In [ ]:
def get_fractions(image):

    ENDMEMBERS = [
        [0.0119,0.0475,0.0169,0.625,0.2399,0.0675], # gv
        [0.1514,0.1597,0.1421,0.3053,0.7707,0.1975], # npv
        [0.1799,0.2479,0.3158,0.5437,0.7707,0.6646], # soil
        [0.4031,0.8714,0.79,0.8989,0.7002,0.6607] # cloud
    ]

    if USE_INT_VALUES:
        ENDMEMBERS = [
            [119.0, 475.0, 169.0, 6250.0, 2399.0, 675.0],  # gv
            [1514.0, 1597.0, 1421.0, 3053.0, 7707.0, 1975.0], # npv
            [1799.0, 2479.0, 3158.0, 5437.0, 7707.0, 6646.0],  # soil
            [4031.0, 8714.0, 7900.0, 8989.0, 7002.0, 6607.0]  # cloud
        ]


    outBandNames = ['gv', 'npv', 'soil', 'cloud']

    fractions = ee.Image(image)\
        .select(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])\
        .unmix(ENDMEMBERS)

    if USE_INT_VALUES:
        fractions = fractions\
            .max(0)\
            .multiply(100)\
            .byte()
    else:
        fractions = fractions\
            .max(0)
            #.multiply(100)\
            #.byte()

    fractions = fractions.rename(outBandNames)

    summed = fractions.expression('b("gv") + b("npv") + b("soil")')

    if USE_INT_VALUES:
        shade = summed.subtract(100).abs().byte().rename("shade")
    else:
        shade = summed.subtract(1).abs().rename("shade")


    fractions = fractions.addBands(shade)

    return image.addBands(fractions)


def get_ndfi(image):

    summed = image.expression('b("gv") + b("npv") + b("soil")')

    if USE_INT_VALUES:
        gvs = image.select("gv").divide(summed).multiply(100).rename("gvs")
    else:
        gvs = image.select("gv").divide(summed).rename("gvs")

    npvSoil = image.expression('b("npv") + b("soil")')

    ndfi = ee.Image.cat(gvs, npvSoil) \
        .normalizedDifference() \
        .rename('ndfi')

    if USE_INT_VALUES:
        # rescale NDFI from 0 to 200 \
        ndfi = ndfi.expression('byte(b("ndfi") * 100 + 100)')

    image = image.addBands(gvs)
    image = image.addBands(ndfi)

    return ee.Image(image)


def get_collection(date_start, date_end, cloud_cover, roi, cloud_thresh):
    collection = None
    l5 = None
    l7 = None
    l8 = None
    l9 = None
    bands = [
        'blue',
        'green',
        'red',
        'nir',
        'swir1',
        'swir2',
        'pixel_qa',
        'tir'
    ]

    l5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
        .filter(ee.Filter.lte('CLOUD_COVER', cloud_cover)) \
        .filterBounds(roi) \
        .filterDate(date_start, date_end)

    l5 = scale_factor_bands(USE_INT_VALUES, l5) \
        .select(
            ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL', 'ST_B6'],
            bands
        ) \
        .map(lambda image: image.set('time', image.get('system:time_start')))

    l7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
        .filter(ee.Filter.lte('CLOUD_COVER', cloud_cover)) \
        .filterBounds(roi) \
        .filterDate(date_start, date_end)

    l7 = scale_factor_bands(USE_INT_VALUES, l7) \
        .select(
            ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL', 'ST_B6'],
            bands
        ) \
        .map(lambda image: image.set('time', image.get('system:time_start')))

    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filter(ee.Filter.lte('CLOUD_COVER', cloud_cover)) \
        .filterBounds(roi) \
        .filterDate(date_start, date_end)

    l8 = scale_factor_bands(USE_INT_VALUES, l8) \
        .select(
            ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL', 'ST_B10'],
            bands
        ) \
        .map(lambda image: image.set('time', image.get('system:time_start')))



    l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
        .filter(ee.Filter.lte('CLOUD_COVER', cloud_cover)) \
        .filterBounds(roi) \
        .filterDate(date_start, date_end)

    l9 = scale_factor_bands(USE_INT_VALUES, l9) \
        .select(
            ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL', 'ST_B10'],
            bands
        ) \
        .map(lambda image: image.set('time', image.get('system:time_start')))


    if SENSOR == 'LANDSAT':
        collection = l5.merge(l7).merge(l8).merge(l9)
    else:
        collection = ee.ImageCollection(ASSET_SENTINEL)\
            .filter(ee.Filter.lte('CLOUD_COVER', cloud_cover)) \
            .filterBounds(roi) \
            .filterDate(date_start, date_end)\
            .linkCollection(cloud_prob_sentinel, [QA_BAND])\
            .map(lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)))

    if USE_INT_VALUES:
        collection = collection \
            .map(get_fractions) \
            .map(get_ndfi) \
            .map(lambda image: image.set('cloudTresh', cloud_thresh)) \
            .map(remove_cloud_shadow)
    else:
        collection = collection \
            .map(get_fractions) \
            .map(get_ndfi) \
            .map(lambda image: image.set('cloudTresh', cloud_thresh)) \
            .map(remove_cloud_shadow)

    return collection


def remove_cloud_shadow(image):
    image = ee.Image(image)
    qa = image.select('qa_pixel')

    if USE_INT_VALUES:
        cloud_threshold = image.select('cloud').lt(ee.Number(image.get('cloudTresh')))
    else:
        cloud_threshold = image.select('cloud').lt(0.025)

    cond = cloud_threshold
    kernel = ee.Kernel.euclidean(60, 'meters')
    proximity = cond.distance(kernel, False)

    cond = cond.where(proximity.gt(0), 0)
    image = image.updateMask(cond)

    kernel = ee.Kernel.euclidean(60, 'meters')
    proximity = image.select('ndfi').unmask(-1).eq(-1).distance(kernel, False)

    cond = cond.where(proximity.gt(0), 0)
    image = image.updateMask(cond)

    return image


def scale_image(image):
    optical_bands = image.select('SR_B.') \
        .multiply(0.0000275) \
        .add(-0.2) \
        .multiply(10000)

    thermal_bands = image.select('ST_B.*') \
        .multiply(0.00341802) \
        .add(149.0) \
        .multiply(10)

    image = image.addBands(optical_bands, None, True)
    image = image.addBands(thermal_bands, None, True)

    return image


def scale_factor_bands(is_int, collection):
    if is_int:
        collection = collection.map(scale_image)
    else:
        collection = collection.map(lambda image: image.multiply(0.0000275).add(-0.2).selfMask().copyProperties(image))

    return collection


## Basic Input Data

In [ ]:
collectionT0 = get_collection(PERIOD_T0[0], PERIOD_T0[1], 50, YOUR_ROI, 0.3)
collectionT0 = collectionT0.select('ndfi')

collectionT1 = get_collection(PERIOD_T1[0], PERIOD_T1[1], 50, YOUR_ROI,  0.3)
collectionT1 = collectionT1.select('ndfi')

# just a median mosaic
mosaic = collectionT1.reduce(ee.Reducer.median())

## Display Data

In [ ]:
palette = 'FFFFFF,FFFCFF,FFF9FF,FFF7FF,FFF4FF,FFF2FF,FFEFFF,FFECFF,FFEAFF,FFE7FF,'+\
  'FFE5FF,FFE2FF,FFE0FF,FFDDFF,FFDAFF,FFD8FF,FFD5FF,FFD3FF,FFD0FF,FFCEFF,'+\
  'FFCBFF,FFC8FF,FFC6FF,FFC3FF,FFC1FF,FFBEFF,FFBCFF,FFB9FF,FFB6FF,FFB4FF,'+\
  'FFB1FF,FFAFFF,FFACFF,FFAAFF,FFA7FF,FFA4FF,FFA2FF,FF9FFF,FF9DFF,FF9AFF,'+\
  'FF97FF,FF95FF,FF92FF,FF90FF,FF8DFF,FF8BFF,FF88FF,FF85FF,FF83FF,FF80FF,'+\
  'FF7EFF,FF7BFF,FF79FF,FF76FF,FF73FF,FF71FF,FF6EFF,FF6CFF,FF69FF,FF67FF,'+\
  'FF64FF,FF61FF,FF5FFF,FF5CFF,FF5AFF,FF57FF,FF55FF,FF52FF,FF4FFF,FF4DFF,'+\
  'FF4AFF,FF48FF,FF45FF,FF42FF,FF40FF,FF3DFF,FF3BFF,FF38FF,FF36FF,FF33FF,'+\
  'FF30FF,FF2EFF,FF2BFF,FF29FF,FF26FF,FF24FF,FF21FF,FF1EFF,FF1CFF,FF19FF,'+\
  'FF17FF,FF14FF,FF12FF,FF0FFF,FF0CFF,FF0AFF,FF07FF,FF05FF,FF02FF,FF00FF,'+\
  'FF00FF,FF0AF4,FF15E9,FF1FDF,FF2AD4,FF35C9,FF3FBF,FF4AB4,FF55AA,FF5F9F,'+\
  'FF6A94,FF748A,FF7F7F,FF8A74,FF946A,FF9F5F,FFAA55,FFB44A,FFBF3F,FFC935,'+\
  'FFD42A,FFDF1F,FFE915,FFF40A,FFFF00,FFFF00,FFFB00,FFF700,FFF300,FFF000,'+\
  'FFEC00,FFE800,FFE400,FFE100,FFDD00,FFD900,FFD500,FFD200,FFCE00,FFCA00,'+\
  'FFC600,FFC300,FFBF00,FFBB00,FFB700,FFB400,FFB000,FFAC00,FFA800,FFA500,'+\
  'FFA500,F7A400,F0A300,E8A200,E1A200,D9A100,D2A000,CA9F00,C39F00,BB9E00,'+\
  'B49D00,AC9C00,A59C00,9D9B00,969A00,8E9900,879900,7F9800,789700,709700,'+\
  '699600,619500,5A9400,529400,4B9300,439200,349100,2D9000,258F00,1E8E00,'+\
  '168E00,0F8D00,078C00,008C00,008C00,008700,008300,007F00,007A00,007600,'+\
  '007200,006E00,006900,006500,006100,005C00,005800,005400,005000,004C00'


Map.centerObject(YOUR_ROI, 10)
Map


Map(center=[-3.0915996970757798, -47.59406515861764], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:

Map.addLayer(mosaic, {
    min:-1,
    max:1,
    palette:['red', 'orange', 'yellow', 'green'] # change this palette
}, 'NDFI')



